In [1]:
import pandas as pd
import json
import copy
json_wb = pd.read_json('world_bank_projects.json')

In [2]:
json_wb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
sector                      500 non-null object
supplementprojectflg        498 non-null object
projectfinancialtype        500 non-null object
prodline                    500 non-null object
mjtheme                     491 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
project_name                500 non-null object
mjthemecode                 500 non-null object
closingdate                 370 non-null object
totalcommamt                500 non-null int64
id                          500 non-null object
mjsector_namecode           500 non-null object
docty                       446 non-null object
sector1                     500 non-null object
lendinginstr                495 non-null object
countrycode                 500 non-null object
sector2                     380 non-null object
totalamt                    500 n

In [3]:
country_project_df = json_wb[['sector', 'countrycode', 'countryshortname', 'country_namecode', 'countryname']]

In [4]:
#1) 10 countries with most projects
country_project_df['countryname'].value_counts().head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

In [5]:
#column of interest for problem #2
project_themes_df = json_wb.loc[:,'mjtheme_namecode']

In [6]:
#2) Top 10 major project themese
project_themes_df.value_counts().head(10)

[{'code': '11', 'name': 'Environment and natural resources management'}, {'code': '11', 'name': 'Environment and natural resources management'}]                                                                                12
[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]                                                                                                                                                        11
[{'code': '8', 'name': 'Human development'}, {'code': '8', 'name': 'Human development'}]                                                                                                                                         8
[{'code': '4', 'name': 'Financial and private sector development'}, {'code': '4', 'name': 'Financial and private sector development'}]                                                                                           6
[{'code': '2', 'name': 'Public sector governance'}, {'code': '2', 'name': 'Public sector gov

In [8]:
#create dictionary to map the project names with their respective codes
mydict = {}
for item in project_themes_df:
    for i in item:
        key = i['code']
        value = i['name']
        if key in mydict.keys():
            pass
        else:
            if i['name'] != '':
                mydict[key] = i['name']

In [9]:
print(mydict)

{'8': 'Human development', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '11': 'Environment and natural resources management', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [10]:
#3) Fills in missing values for name in column 'mjtheme_namecode'
json_wb['new_col'] = [[i if i['name'] else {'name', mydict[i['code']]} for i in item]
                     for item in json_wb['mjtheme_namecode']]

In [11]:
#shows the differences between the new column and the original 'mjtheme_namecode'. this is all rows where there was 
#an entry into the 'name' field in the original column
json_wb[json_wb['new_col'] != json_wb['mjtheme_namecode']]

,sector,supplementprojectflg,projectfinancialtype,prodline,mjtheme,idacommamt,impagency,project_name,mjthemecode,closingdate,...,board_approval_month,theme_namecode,countryname,url,source,projectstatusdisplay,ibrdcommamt,sector_namecode,_id,new_col
0,"[{'Name': 'Primary education'}, {'Name': 'Seco...",N,IDA,PE,[Human development],130000000,MINISTRY OF EDUCATION,Ethiopia General Education Quality Improvement...,"8,11",2018-07-07T00:00:00Z,...,November,"[{'code': '65', 'name': 'Education for all'}]",Federal Democratic Republic of Ethiopia,http://www.worldbank.org/projects/P129828/ethi...,IBRD,Active,0,"[{'code': 'EP', 'name': 'Primary education'}, ...",{'$oid': '52b213b38594d8a2be17c780'},"[{'code': '8', 'name': 'Human development'}, {..."
5,[{'Name': 'Other social services'}],Y,IDA,PE,[Social protection and risk management],10000000,NaN,Additional Financing for Cash Transfers for Or...,"6,6",NaN,...,October,"[{'code': '54', 'name': 'Social safety nets'}]",Republic of Kenya,http://www.worldbank.org/projects/P146161?lang=en,IBRD,Active,0,"[{'code': 'JB', 'name': 'Other social services'}]",{'$oid': '52b213b38594d8a2be17c785'},"[{'code': '6', 'name': 'Social protection and ..."
7,[{'Name': 'Other Renewable Energy'}],N,OTHER,GE,[Environment and natural resources management],0,NATIONAL ENERGY ADMINISTRATION,China Renewable Energy Scale-Up Program Phase II,"11,8",NaN,...,October,"[{'code': '81', 'name': 'Climate change'}]",People's Republic of China,http://www.worldbank.org/projects/P127033/chin...,IBRD,Active,0,"[{'code': 'LR', 'name': 'Other Renewable Energ...",{'$oid': '52b213b38594d8a2be17c787'},"[{'code': '11', 'name': 'Environment and natur..."
8,[{'Name': 'Rural and Inter-Urban Roads and Hig...,N,IDA,PE,[Rural development],160000000,"PUBLIC WORKS DEPARTMANT, RAJASTHAN",Rajasthan Road Sector Modernization Project,"10,7",2018-12-31T00:00:00Z,...,October,"[{'code': '79', 'name': 'Other rural developme...",Republic of India,http://www.worldbank.org/projects/P130164/raja...,IBRD,Active,0,"[{'code': 'TI', 'name': 'Rural and Inter-Urban...",{'$oid': '52b213b38594d8a2be17c788'},"[{'code': '10', 'name': 'Rural development'}, ..."
10,"[{'Name': 'Crops'}, {'Name': 'Other social ser...",Y,OTHER,RE,[Rural development],0,"MINISTRY OF AGRICULTURE, COOPERATIVES AND RURA...",Southern Sudan Emergency Food Crisis Response ...,"10,2",NaN,...,October,"[{'code': '91', 'name': 'Global food crisis re...",Republic of South Sudan,http://www.worldbank.org/projects/P145339?lang=en,IBRD,Active,0,"[{'code': 'AH', 'name': 'Crops'}, {'code': 'JB...",{'$oid': '52b213b38594d8a2be17c78a'},"[{'code': '10', 'name': 'Rural development'}, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,"[{'Name': 'Animal production'}, {'Name': 'Heal...",N,OTHER,RE,[Human development],0,"NATIONAL EMERGENCY MANAGEMENT AGENCY, MOH, MOFALI",Capacity Builiding for Emerging Infectious Dis...,"8,4",2013-12-31T00:00:00Z,...,September,"[{'code': '64', 'name': 'Other communicable di...",Mongolia,http://www.worldbank.org/projects/P131204/capa...,IBRD,Active,0,"[{'code': 'AJ', 'name': 'Animal production'}, ...",{'$oid': '52b213b38594d8a2be17c965'},"[{'code': '8', 'name': 'Human development'}, {..."
491,"[{'Name': 'Sanitation'}, {'Name': 'Water suppl...",N,OTHER,RE,[Rural development],0,JOINT SERVICES COUNCIL,Water Supply and Sanitation Improvements for W...,"10,11",2014-03-31T00:00:00Z,...,August,"[{'code': '78', 'name': 'Rural services and in...",West Bank and Gaza,http://www.worldbank.org/projects/P123322/wate...,IBRD,Active,0,"[{'code': 'WA', 'name': 'Sanitation'}, {'code'...",{'$oid': '52b213b38594d8a2be17c96b'},"[{'code': '10', 'name': 'Rural development'}, ..."
493,[{'Name': 'Central government administration'}],N,OTHER,RE,[Environment and natural resources management],0,GEF SECRETARIAT,Revision and Alignment of NAP with UNCCD 10-ye...,"11,5",2015-04-30T00:00:00Z,...,August,"[{'code': '86', 'name': 'Other environment and...",People's Republ